In [1]:
# from pyspark import SparkFiles
import twitter
import os
from config import *
import pandas as pd
import numpy as np


# initialize api instance
twitter_api = twitter.Api(consumer_key=consumer_key,
                         consumer_secret=consumer_secret,
                         access_token_key=access_token_key,
                         access_token_secret=access_token_secret)

#print(consumer_key,consumer_secret,access_token_key,access_token_secret)

# test authentication
print(twitter_api.VerifyCredentials())

{"created_at": "Wed Jul 20 02:34:27 +0000 2016", "description": "I am Al and I am passionate about technology.", "followers_count": 5, "friends_count": 52, "id": 755591669194514432, "id_str": "755591669194514432", "location": "Tustin, CA", "name": "Al", "profile_background_color": "000000", "profile_background_image_url": "http://abs.twimg.com/images/themes/theme1/bg.png", "profile_background_image_url_https": "https://abs.twimg.com/images/themes/theme1/bg.png", "profile_banner_url": "https://pbs.twimg.com/profile_banners/755591669194514432/1469041661", "profile_image_url": "http://pbs.twimg.com/profile_images/755843006062694401/2JikSpJq_normal.jpg", "profile_image_url_https": "https://pbs.twimg.com/profile_images/755843006062694401/2JikSpJq_normal.jpg", "profile_link_color": "1B95E0", "profile_sidebar_border_color": "000000", "profile_sidebar_fill_color": "000000", "profile_text_color": "000000", "screen_name": "alknowstech", "status": {"created_at": "Mon Jul 31 21:38:46 +0000 2017", 

# Only run if configuring spark

In [ ]:
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

# Get the default configurations
spark.sparkContext._conf.getAll()

# Update the default configurations
conf = spark.sparkContext._conf.setAll([("spark.executor.heartbeatInterval", "300000"),
                                       ("spark.network.timeout", "400000"),('spark.executor.memory', '6g'),
                                        ('spark.app.name', 'Spark Updated Conf'),
                                        ('spark.executor.cores', '6'), ('spark.cores.max', '6'), 
                                        ('spark.driver.memory','6g')])

# Stop the current Spark Session
spark.sparkContext.stop()

# Create a Spark Session
spark = SparkSession.builder.config(conf=conf).getOrCreate()


In [2]:
traindata140 = os.path.join(".","trainingandtestdata_sentiment140","training.csv")

train_df = pd.read_csv(traindata140, header=None, usecols=[0,5], names=['polarity of the tweet', 'text'], 
                       encoding="ISO-8859-1")
train_df.head()
# (0 = negative, 2 = neutral, 4 = positive)

,polarity of the tweet,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [3]:
# Test Data
testdata140 = os.path.join(".","trainingandtestdata_sentiment140","test.csv")

test_df = pd.read_csv(testdata140, header=None, usecols=[0,5], names=['polarity of the tweet', 'text'], 
                       encoding="ISO-8859-1")
test_df.head()
# (0 = negative, 2 = neutral, 4 = positive)

conditions_test = [
    (test_df['polarity of the tweet'] == 0),
    (test_df['polarity of the tweet'] == 2),
    (test_df['polarity of the tweet'] == 4)]
choices_test = ['negative', 'neutral', 'positive']
test_df['class'] = np.select(conditions_test, choices_test)
test_df.head()

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

# convert pandas DF to spark Df
spark_df_test = sqlContext.createDataFrame(test_df.iloc[:,1:3])
spark_df_test.show()

+--------------------+--------+
|                text|   class|
+--------------------+--------+
|@stellargirl I lo...|positive|
|Reading my kindle...|positive|
|Ok, first assesme...|positive|
|@kenburbary You'l...|positive|
|@mikefish  Fair e...|positive|
|@richardebaker no...|positive|
|Fuck this economy...|negative|
|Jquery is my new ...|positive|
|       Loves twitter|positive|
|how can you not l...|positive|
|Check this video ...| neutral|
|@Karoli I firmly ...|negative|
|House Corresponde...|positive|
|Watchin Espn..Jus...|positive|
|dear nike, stop w...|negative|
|#lebron best athl...|positive|
|I was talking to ...|negative|
|i love lebron. ht...|positive|
|@ludajuice Lebron...|negative|
|@Pmillzz lebron I...|positive|
+--------------------+--------+
only showing top 20 rows



In [4]:
conditions = [
    (train_df['polarity of the tweet'] == 0),
    (train_df['polarity of the tweet'] == 2),
    (train_df['polarity of the tweet'] == 4)]
choices = ['negative', 'neutral', 'positive']
train_df['class'] = np.select(conditions, choices)
train_df.head()

,polarity of the tweet,text,class
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",negative
1,0,is upset that he can't update his Facebook by ...,negative
2,0,@Kenichan I dived many times for the ball. Man...,negative
3,0,my whole body feels itchy and like its on fire,negative
4,0,"@nationwideclass no, it's not behaving at all....",negative


In [5]:
train_ls = train_df.to_dict('records')

In [6]:
import re
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 
import nltk
# nltk.download('punkt')

class PreProcessTweets:
    def __init__(self):
        self._stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
        
    def processTweets(self, list_of_tweets):
        processedTweets=[]
        for tweet in list_of_tweets:
            processedTweets.append((self._processTweet(tweet["text"]),tweet["class"],tweet["text"]))
        return processedTweets
    
    def _processTweet(self, tweet):
        tweet = tweet.lower() # convert text to lower-case
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
        tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
        return [word for word in tweet if word not in self._stopwords]
    
tweetProcessor = PreProcessTweets()


In [7]:
# Process Training Set
preprocessedTrainingSet = tweetProcessor.processTweets(train_ls)

In [8]:
preprocessedTrainingSet

[(['awww', "'s", 'bummer', 'shoulda', 'got', 'david', 'carr', 'third', 'day'],
  'negative',
  "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"),
 (['upset',
   'ca',
   "n't",
   'update',
   'facebook',
   'texting',
   '...',
   'might',
   'cry',
   'result',
   'school',
   'today',
   'also',
   'blah'],
  'negative',
  "is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"),
 (['dived',
   'many',
   'times',
   'ball',
   'managed',
   'save',
   '50',
   'rest',
   'go',
   'bounds'],
  'negative',
  '@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds'),
 (['whole', 'body', 'feels', 'itchy', 'like', 'fire'],
  'negative',
  'my whole body feels itchy and like its on fire '),
 (["'s", 'behaving', "'m", 'mad', 'ca', "n't", 'see'],
  'negative',
  "@nationwideclass no, it's not behaving at all. i'm mad. why am i her

In [9]:
train_df= pd.DataFrame(preprocessedTrainingSet, columns = ['tokenized' , 'class','text']) 

In [10]:
train_df.head()

,tokenized,class,text
0,"[awww, 's, bummer, shoulda, got, david, carr, ...",negative,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,"[upset, ca, n't, update, facebook, texting, .....",negative,is upset that he can't update his Facebook by ...
2,"[dived, many, times, ball, managed, save, 50, ...",negative,@Kenichan I dived many times for the ball. Man...
3,"[whole, body, feels, itchy, like, fire]",negative,my whole body feels itchy and like its on fire
4,"['s, behaving, 'm, mad, ca, n't, see]",negative,"@nationwideclass no, it's not behaving at all...."


In [11]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

# convert pandas DF to spark Df
spark_df = sqlContext.createDataFrame(train_df)
spark_df.show()

+--------------------+--------+--------------------+
|           tokenized|   class|                text|
+--------------------+--------+--------------------+
|[awww, 's, bummer...|negative|@switchfoot http:...|
|[upset, ca, n't, ...|negative|is upset that he ...|
|[dived, many, tim...|negative|@Kenichan I dived...|
|[whole, body, fee...|negative|my whole body fee...|
|['s, behaving, 'm...|negative|@nationwideclass ...|
|       [whole, crew]|negative|@Kwesidei not the...|
|         [need, hug]|negative|         Need a hug |
|[hey, long, time,...|negative|@LOLTrish hey  lo...|
|         [nope, n't]|negative|@Tatiana_K nope t...|
|        [que, muera]|negative|@twittera que me ...|
|[spring, break, p...|negative|spring break in p...|
|  [re-pierced, ears]|negative|I just re-pierced...|
|[could, n't, bear...|negative|@caregiving I cou...|
|[counts, idk, eit...|negative|@octolinz16 It it...|
|[would, 've, firs...|negative|@smarrison i woul...|
|[wish, got, watch...|negative|@iamjazzyfizzle

In [12]:
from pyspark.sql.functions import length
# Create a length column to be used as a future feature 
data_df = spark_df.withColumn('length', length(spark_df['text']))
data_df.show()

+--------------------+--------+--------------------+------+
|           tokenized|   class|                text|length|
+--------------------+--------+--------------------+------+
|[awww, 's, bummer...|negative|@switchfoot http:...|   115|
|[upset, ca, n't, ...|negative|is upset that he ...|   111|
|[dived, many, tim...|negative|@Kenichan I dived...|    89|
|[whole, body, fee...|negative|my whole body fee...|    47|
|['s, behaving, 'm...|negative|@nationwideclass ...|   111|
|       [whole, crew]|negative|@Kwesidei not the...|    29|
|         [need, hug]|negative|         Need a hug |    11|
|[hey, long, time,...|negative|@LOLTrish hey  lo...|    99|
|         [nope, n't]|negative|@Tatiana_K nope t...|    36|
|        [que, muera]|negative|@twittera que me ...|    25|
|[spring, break, p...|negative|spring break in p...|    43|
|  [re-pierced, ears]|negative|I just re-pierced...|    26|
|[could, n't, bear...|negative|@caregiving I cou...|    94|
|[counts, idk, eit...|negative|@octolinz

### Feature Transformations


In [13]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer #,RegexTokenizer
# Create all the features to the data set
pos_neg_to_num = StringIndexer(inputCol='class',outputCol='label')
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='tokenized',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="tokenized", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')


# pos_neg_to_num = StringIndexer(inputCol='class',outputCol='label')
# tokenizer = Tokenizer(inputCol=regex, outputCol="token_text")
# regex = RegexTokenizer(inputCol="text", outputCol="regex_text", pattern='((www\.[^\s]+)|(https?://[^\s]+))|@[^\s]+|#([^\s]+)')
# stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
# hashingTF = HashingTF(inputCol="token_text", outputCol='hash_token')
# idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [14]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [15]:
# Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[pos_neg_to_num, tokenizer, 
                                      stopremove, hashingTF, idf, clean_up])

In [ ]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(data_df)
cleaned = cleaner.transform(data_df)

In [ ]:
cleaned.show(1)

In [ ]:
# Show label and resulting features
cleaned.select(['label', 'features']).show()

In [ ]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
# training, testing = cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(cleaned)

In [ ]:
# Tranform the model with the testing data
test_results = predictor.transform(testing)
test_results.show(5)

In [ ]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" % acc)